In [49]:
import os
from dotenv import load_dotenv

load_dotenv()

CLAUDE_KEY = os.getenv("CLAUDE_KEY")
#MODEL = "claude-3-7-sonnet-20250219"
MODEL = "llama3.1"


In [50]:
from langchain_anthropic import ChatAnthropic
from langchain_ollama.llms import OllamaLLM
from langchain_ollama import OllamaEmbeddings


if MODEL.startswith("claude"):
    llm = ChatAnthropic(model=MODEL, api_key=CLAUDE_KEY)
else:
    llm = OllamaLLM(model=MODEL)
    embeddings = OllamaEmbeddings(model=MODEL)
    

    
llm.invoke("tell me a joke")

"Here's one:\n\nWhat do you call a fake noodle?\n\nAn impasta!\n\nHope that made you laugh! Do you want to hear another?"

In [51]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = llm | parser
chain.invoke("tell me a joke")

"Here's one:\n\nWhat do you call a fake noodle?\n\nAn impasta!"

In [63]:
from docling.document_converter import DocumentConverter
from langchain.schema import Document
import os

converter = DocumentConverter()

def load_pdfs(path):
    documents = []
    ctr = 1

    for filename in os.listdir(path):
        if filename.endswith(".pdf"):
            file_path = os.path.join(path, filename)

            result = converter.convert(file_path)
            content = result.document.export_to_markdown()

            doc = Document(
                page_content = content,
                metadata = {"source": file_path}
            )

            print(f"File {ctr}: {filename} loaded.")
            ctr += 1
            documents.append(doc)
    return documents


In [64]:
all_documents = load_pdfs("data/preprocessed")
len(all_documents)

File 1: localsearch.pdf loaded.
File 2: informed_search.pdf loaded.
File 3: naivebayes.pdf loaded.
File 4: adversarialsearch.pdf loaded.
File 5: Search.pdf loaded.
File 6: intelligentagents.pdf loaded.
File 7: markov.pdf loaded.
File 8: rl.pdf loaded.
File 9: ml.pdf loaded.
File 10: AI_Intro.pdf loaded.


10

In [65]:
def export_markdown_files(documents, output_dir="data/processed_markdown"):
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    for doc in documents:
        # Get the original filename from the source path
        file_path = doc.metadata["source"]
        base_name = os.path.splitext(os.path.basename(file_path))[0]
        
        # Create markdown file path
        markdown_path = os.path.join(output_dir, f"{base_name}.md")
        
        # Write content to markdown file
        with open(markdown_path, 'w', encoding='utf-8') as f:
            f.write(doc.page_content)
        
        # Update metadata to include markdown path
        doc.metadata["markdown_path"] = markdown_path
        
        print(f"Exported: {markdown_path}")
    
    return documents

In [66]:
all_documents = export_markdown_files(all_documents)


Exported: data/processed_markdown/localsearch.md
Exported: data/processed_markdown/informed_search.md
Exported: data/processed_markdown/naivebayes.md
Exported: data/processed_markdown/adversarialsearch.md
Exported: data/processed_markdown/Search.md
Exported: data/processed_markdown/intelligentagents.md
Exported: data/processed_markdown/markov.md
Exported: data/processed_markdown/rl.md
Exported: data/processed_markdown/ml.md
Exported: data/processed_markdown/AI_Intro.md


In [34]:
from langchain.prompts import PromptTemplate

template = """Answer the question based on the context below. If you can't answer the question, reply "I do not know"

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="here is a question") )

Answer the question based on the context below. If you can't answer the question, reply "I do not know"

Context: Here is some context

Question: here is a question



In [35]:
chain = prompt | llm | parser 

In [36]:
chain.invoke(
    {
        "context": "The name I was given was Ant Man",
        "question": "What is my name?",
    }
)

'Your name is Ant-Man.'

In [37]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    pages, 
    embedding=embeddings
)

/Users/aashishd/Desktop/llama-rag/.venv/lib/python3.9/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [40]:
retriever = vectorstore.as_retriever()

retriever.invoke("Rational Agent")

[Document(metadata={'producer': 'Acrobat Distiller 11.0 (Windows)', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2022-01-31T17:50:48-06:00', 'author': 'Weihua', 'moddate': '2022-01-31T17:50:48-06:00', 'title': 'Microsoft PowerPoint - m2-agents.ppt [Compatibility Mode]', 'source': 'data/preprocessed/intelligentagents.pdf', 'total_pages': 27, 'page': 0, 'page_label': '1'}, page_content='Intelligent Agents\nChapter 2'),
 Document(metadata={'producer': 'Acrobat Distiller 11.0 (Windows)', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2022-01-31T17:50:48-06:00', 'author': 'Weihua', 'moddate': '2022-01-31T17:50:48-06:00', 'title': 'Microsoft PowerPoint - m2-agents.ppt [Compatibility Mode]', 'source': 'data/preprocessed/intelligentagents.pdf', 'total_pages': 27, 'page': 25, 'page_label': '26'}, page_content='Utility-based agents\nENVTrigger/Prioritize\nGoals/Tasks\nGoals Tasks\nPercepts\nActions\nUpdate \nWorld \nModel\nWorld Model\nProblem Solving Methods\nSelect \ngo

In [47]:
from operator import itemgetter 

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    }
    | prompt
    | llm
    | parser
)

chain.invoke({"question": "what is PEAS"})

"PEAS stands for Performance measure, Environment, Actuators, Sensors. It's mentioned in the third document as one of the components of an intelligent agent, described in the outline on page 2."

In [48]:
for s in chain.stream({"question": "What is PEAS?"}):
    print(s, end="", flush=True)

PEAS stands for:

• E (Environment): A description of the external world that the agent interacts with.
• A (Actuators): The agents capabilities to interact with its environment by performing actions. 
• S (Sensors): The information the agent receives from its sensors about its environment and itself.
• P (Performance Measure): The criteria for measuring how well or poorly an intelligent agent is doing.

This is explained in more detail on page 14 of the document, as mentioned in the context provided.

In [ ]:
#next steps:

'''
all the ai pdfs
using docling for pdf handling 
scale it up
make the overview pdf 
start the second project
'''